In [3]:
import time
import pandas as pd
import json
import logging
from tqdm import tqdm
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------- Настройка логирования ----------
logging.basicConfig(
    filename='inference_log.txt',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


my_max_tokens = 2000

# ---------- Проверка и настройка устройства ----------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA недоступна. Убедитесь, что GPU доступна и PyTorch установлен с поддержкой CUDA.")

device = torch.device("cuda")
torch_dtype = torch.float16

print("CUDA доступна:", torch.cuda.is_available())
print("Количество доступных GPU:", torch.cuda.device_count())
print("Текущее устройство:", torch.cuda.current_device())
print("Имя устройства:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ---------- Показ GPU памяти ----------
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"🧠 Использовано GPU памяти: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"🛡 Зарезервировано GPU памяти: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
    else:
        print("CUDA недоступна.")

torch.cuda.empty_cache()

# ---------- Время начала выполнения ----------
start_global_time = time.time()

# ---------- Параметры входных и выходных файлов ----------
input_file = '/home/skoltsov/Open-Reasoner-Zero-1.5B/equal_data/test_x_balans.xlsx'
output_json = r'/home/skoltsov/Open-Reasoner-Zero-1.5B/inference_results/results_Open-Reasoner-Zero_2000tok.json'
data_fraction = 1.0  # Доля данных для обработки

# ---------- Чтение данных ----------
df = pd.read_excel(input_file)
df = df.sample(frac=data_fraction, random_state=42).reset_index(drop=True)

# ---------- Загрузка модели и токенизатора ----------
model_path = r"/home/skoltsov/Open-Reasoner-Zero-1.5B/Open-Reasoner-Zero-1.5B_model"

start_load_time = time.time()

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch_dtype
    ).to(device)
    logging.info("Модель и токенизатор успешно загружены.")
except Exception as e:
    logging.error(f"Ошибка при загрузке модели или токенизатора: {e}")
    print(f"Ошибка при загрузке модели или токенизатора: {e}")
    exit()

end_load_time = time.time()
load_time = end_load_time - start_load_time
print(f"⏱️ Время загрузки модели и данных: {load_time:.2f} секунд")

# ---------- Функция генерации решения ----------
def solve_differential_equation(prompt, temperature=0.2, max_new_tokens=my_max_tokens):
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate( **inputs, max_new_tokens=max_new_tokens, do_sample=False, temperature=temperature, pad_token_id=tokenizer.eos_token_id)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return output_text
    except Exception as e:
        logging.error(f"Ошибка генерации: {e}")
        print(f"Ошибка при решении уравнения: {e}")
        return None

# ---------- Основной цикл инференса ----------
results = []
row_times = []
total_rows = len(df)
processed_rows = 0

for _, row in tqdm(df.iterrows(), total=total_rows, desc="Обработка уравнений"):
    start_row_time = time.time()

    try:
        true_answer = row['true_answer']
        equation = row['equation']
        equation_type = row['type_eq']

        prompt = (
            f"Solve the differential equation: {equation}. "
            f"It must be provided in the final answer in LaTeX format, enclosed in \\boxed{{}}"
        )

        logging.info(f"Обрабатывается уравнение: {equation}")
        response = solve_differential_equation(prompt)

        results.append({
            "equation": equation,
            "true_answer": true_answer,
            "model_response": response,
            "equation_type": equation_type
        })

    except Exception as e:
        logging.warning(f"Ошибка при обработке уравнения: {equation} — {e}")
        results.append({
            "equation": equation,
            "true_answer": None,
            "model_response": None,
            "equation_type": "Неизвестный тип"
        })

    processed_rows += 1
    tqdm.write(f"Обработано: {processed_rows}/{total_rows} ({(processed_rows / total_rows) * 100:.2f}%)")

    row_times.append(time.time() - start_row_time)

# ---------- Сохранение результатов ----------
try:
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f"✅ Результаты сохранены в {output_json}")
except Exception as e:
    logging.error(f"Ошибка при сохранении JSON: {e}")
    print(f"Ошибка при сохранении JSON: {e}")

# ---------- Вывод статистики ----------
end_global_time = time.time()
total_time = end_global_time - start_global_time

print("\n📊 Статистика времени:")
print(f"⏱️ Общее время выполнения: {total_time:.2f} секунд")
print(f"⏱️ Время загрузки модели: {load_time:.2f} секунд")
print(f"⏱️ Среднее время на строку: {sum(row_times)/len(row_times):.2f} секунд")
print(f"⏱️ Всего строк: {total_rows}, успешно обработано: {processed_rows}")

print_gpu_memory()


CUDA доступна: True
Количество доступных GPU: 1
Текущее устройство: 0
Имя устройства: NVIDIA A100-SXM4-80GB
⏱️ Время загрузки модели и данных: 1.48 секунд


Обработка уравнений:   0%|          | 1/1710 [00:22<10:42:45, 22.57s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Обработано: 1/1710 (0.06%)


Обработка уравнений:   0%|          | 2/1710 [01:15<19:10:37, 40.42s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Обработано: 2/1710 (0.12%)


...

Обработано: 4/1710 (0.23%)


Обработка уравнений:   0%|          | 5/1710 [02:19<11:39:57, 24.63s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Обработано: 5/1710 (0.29%)


Обработка уравнений:   0%|          | 5/1710 [02:30<14:16:02, 30.12s/it]


KeyboardInterrupt: 

In [5]:
import time
import pandas as pd
import json
import logging
from tqdm import tqdm
import os
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------- Настройка логирования ----------
logging.basicConfig(
    filename='inference_log.txt',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

my_max_tokens = 2000
batch_size = 82  # ⬅️ Можно увеличить, если хватает видеопамяти

# ---------- Проверка и настройка устройства ----------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA недоступна. Убедитесь, что GPU доступна и PyTorch установлен с поддержкой CUDA.")

device = torch.device("cuda")
torch_dtype = torch.float16

print("CUDA доступна:", torch.cuda.is_available())
print("Количество доступных GPU:", torch.cuda.device_count())
print("Текущее устройство:", torch.cuda.current_device())
print("Имя устройства:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ---------- Показ GPU памяти ----------
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"🧠 Использовано GPU памяти: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"🛡 Зарезервировано GPU памяти: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
    else:
        print("CUDA недоступна.")

torch.cuda.empty_cache()

# ---------- Время начала выполнения ----------
start_global_time = time.time()

# ---------- Параметры входных и выходных файлов ----------
input_file = '/home/skoltsov/Open-Reasoner-Zero-1.5B/equal_data/test_x_balans.xlsx'
output_json = r'/home/skoltsov/Open-Reasoner-Zero-1.5B/inference_results/results_Open-Reasoner-Zero_2000tok.json'
data_fraction = 1.0  # Доля данных для обработки

# ---------- Чтение данных ----------
df = pd.read_excel(input_file)
df = df.sample(frac=data_fraction, random_state=42).reset_index(drop=True)

# ---------- Загрузка модели и токенизатора ----------
model_path = r"/home/skoltsov/Open-Reasoner-Zero-1.5B/Open-Reasoner-Zero-1.5B_model"

start_load_time = time.time()

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch_dtype
    ).to(device)
    logging.info("Модель и токенизатор успешно загружены.")
except Exception as e:
    logging.error(f"Ошибка при загрузке модели или токенизатора: {e}")
    print(f"Ошибка при загрузке модели или токенизатора: {e}")
    exit()

end_load_time = time.time()
load_time = end_load_time - start_load_time
print(f"⏱️ Время загрузки модели и данных: {load_time:.2f} секунд")

# ---------- Функция батчевой генерации ----------
def batch_generate(prompts, batch_size=8):
    responses = []
    dataloader = DataLoader(prompts, batch_size=batch_size)

    for batch in tqdm(dataloader, desc="Батчи"):
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=my_max_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        decoded = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        responses.extend(decoded)

    return responses

# ---------- Подготовка промптов ----------
prompts = [
    f"Solve the differential equation: {eq}. Final answer in LaTeX: \\boxed{{}}"
    for eq in df['equation']
]

# ---------- Выполнение инференса батчами ----------
print(f"\n🔄 Начинаем генерацию ответов батчами (размер батча: {batch_size})")
start_inference_time = time.time()

try:
    all_responses = batch_generate(prompts, batch_size=batch_size)
except Exception as e:
    logging.error(f"Ошибка при батчевой генерации: {e}")
    print(f"Ошибка при батчевой генерации: {e}")
    exit()

end_inference_time = time.time()
inference_time = end_inference_time - start_inference_time

# ---------- Сбор результатов ----------
results = []
for i, row in df.iterrows():
    results.append({
        "equation": row['equation'],
        "true_answer": row['true_answer'],
        "model_response": all_responses[i],
        "equation_type": row['type_eq']
    })

# ---------- Сохранение результатов ----------
try:
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f"✅ Результаты сохранены в {output_json}")
except Exception as e:
    logging.error(f"Ошибка при сохранении JSON: {e}")
    print(f"Ошибка при сохранении JSON: {e}")

# ---------- Вывод статистики ----------
end_global_time = time.time()
total_time = end_global_time - start_global_time

print("\n📊 Статистика времени:")
print(f"⏱️ Общее время выполнения: {total_time:.2f} секунд")
print(f"⏱️ Время загрузки модели: {load_time:.2f} секунд")
print(f"⏱️ Время инференса: {inference_time:.2f} секунд")
print(f"⏱️ Среднее время на строку: {inference_time / len(df):.2f} секунд")
print(f"⏱️ Всего строк: {len(df)}")

print_gpu_memory()

CUDA доступна: True
Количество доступных GPU: 1
Текущее устройство: 0
Имя устройства: NVIDIA A100-SXM4-80GB
⏱️ Время загрузки модели и данных: 1.82 секунд

🔄 Начинаем генерацию ответов батчами (размер батча: 82)


Батчи:  10%|▉         | 2/21 [04:20<41:15, 130.29s/it]


KeyboardInterrupt: 

In [8]:
import time
import pandas as pd
import json
import logging
from tqdm import tqdm
import os
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------- Настройка логирования ----------
logging.basicConfig(
    filename='inference_log.txt',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

my_max_tokens = 3000
batch_size = 84  # Можно увеличить при наличии памяти на GPU

# ---------- Проверка и настройка устройства ----------
if not torch.cuda.is_available():
    raise RuntimeError("CUDA недоступна. Убедитесь, что GPU доступна и PyTorch установлен с поддержкой CUDA.")

device = torch.device("cuda")
torch_dtype = torch.float16

print("CUDA доступна:", torch.cuda.is_available())
print("Количество доступных GPU:", torch.cuda.device_count())
print("Текущее устройство:", torch.cuda.current_device())
print("Имя устройства:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ---------- Показ GPU памяти ----------
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"🧠 Использовано GPU памяти: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"🛡 Зарезервировано GPU памяти: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
    else:
        print("CUDA недоступна.")

torch.cuda.empty_cache()

# ---------- Время начала выполнения ----------
start_global_time = time.time()

# ---------- Параметры входных и выходных файлов ----------
input_file = '/home/skoltsov/Open-Reasoner-Zero-1.5B/equal_data/test_x_balans.xlsx'
output_json = r'/home/skoltsov/Open-Reasoner-Zero-1.5B/inference_results/results_Open-Reasoner-Zero_3000tok.json'
data_fraction = 1.0  # Доля данных для обработки

# ---------- Чтение данных ----------
df = pd.read_excel(input_file)
df = df.sample(frac=data_fraction, random_state=42).reset_index(drop=True)

# ---------- Загрузка модели и токенизатора ----------
model_path = r"/home/skoltsov/Open-Reasoner-Zero-1.5B/Open-Reasoner-Zero-1.5B_model"

start_load_time = time.time()

try:
    # Установите padding_side='left' здесь:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, padding_side='left')
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch_dtype
    ).to(device)
    logging.info("Модель и токенизатор успешно загружены.")
except Exception as e:
    logging.error(f"Ошибка при загрузке модели или токенизатора: {e}")
    print(f"Ошибка при загрузке модели или токенизатора: {e}")
    exit()

end_load_time = time.time()
load_time = end_load_time - start_load_time
print(f"⏱️ Время загрузки модели и данных: {load_time:.2f} секунд")

# ---------- Функция батчевой генерации ----------
def batch_generate(prompts, batch_size=8):
    responses = []
    dataloader = DataLoader(prompts, batch_size=batch_size)

    for batch in tqdm(dataloader, desc="Батчи"):
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=my_max_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        decoded = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        responses.extend(decoded)

    return responses

# ---------- Подготовка промптов ----------
prompts = [
    f"Solve the differential equation: {eq}. Final answer in LaTeX: \\boxed{{}}"
    for eq in df['equation']
]

# ---------- Выполнение инференса батчами ----------
print(f"\n🔄 Начинаем генерацию ответов батчами (размер батча: {batch_size})")
start_inference_time = time.time()

try:
    all_responses = batch_generate(prompts, batch_size=batch_size)
except Exception as e:
    logging.error(f"Ошибка при батчевой генерации: {e}")
    print(f"Ошибка при батчевой генерации: {e}")
    exit()

end_inference_time = time.time()
inference_time = end_inference_time - start_inference_time

# ---------- Сбор результатов ----------
results = []
for i, row in df.iterrows():
    results.append({
        "equation": row['equation'],
        "true_answer": row['true_answer'],
        "model_response": all_responses[i],
        "equation_type": row['type_eq']
    })

# ---------- Сохранение результатов ----------
try:
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f"✅ Результаты сохранены в {output_json}")
except Exception as e:
    logging.error(f"Ошибка при сохранении JSON: {e}")
    print(f"Ошибка при сохранении JSON: {e}")

# ---------- Вывод статистики ----------
end_global_time = time.time()
total_time = end_global_time - start_global_time

print("\n📊 Статистика времени:")
print(f"⏱️ Общее время выполнения: {total_time:.2f} секунд")
print(f"⏱️ Время загрузки модели: {load_time:.2f} секунд")
print(f"⏱️ Время инференса: {inference_time:.2f} секунд")
print(f"⏱️ Среднее время на строку: {inference_time / len(df):.2f} секунд")
print(f"⏱️ Всего строк: {len(df)}")

print_gpu_memory()

CUDA доступна: True
Количество доступных GPU: 1
Текущее устройство: 0
Имя устройства: NVIDIA A100-SXM4-80GB
⏱️ Время загрузки модели и данных: 3.13 секунд

🔄 Начинаем генерацию ответов батчами (размер батча: 84)


Батчи: 100%|██████████| 21/21 [1:14:49<00:00, 213.78s/it]


✅ Результаты сохранены в /home/skoltsov/Open-Reasoner-Zero-1.5B/inference_results/results_Open-Reasoner-Zero_3000tok.json

📊 Статистика времени:
⏱️ Общее время выполнения: 4496.39 секунд
⏱️ Время загрузки модели: 3.13 секунд
⏱️ Время инференса: 4489.41 секунд
⏱️ Среднее время на строку: 2.63 секунд
⏱️ Всего строк: 1710
🧠 Использовано GPU памяти: 7275.30 MB
🛡 Зарезервировано GPU памяти: 48594.00 MB


In [ ]:
✅ Результаты сохранены в /home/skoltsov/Open-Reasoner-Zero-1.5B/inference_results/results_Open-Reasoner-Zero_2000tok.json

📊 Статистика времени:
⏱️ Общее время выполнения: 2540.43 секунд
⏱️ Время загрузки модели: 1.45 секунд
⏱️ Время инференса: 2538.64 секунд
⏱️ Среднее время на строку: 1.48 секунд
⏱️ Всего строк: 1710
🧠 Использовано GPU памяти: 15076.48 MB
🛡 Зарезервировано GPU памяти: 51462.00 MB


42.3405